In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
!pip install thop

In [19]:
import torch
import torch.nn as nn
import time
import os
from torchvision import models
from thop import profile
import random
import numpy as np

In [20]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [21]:
set_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [22]:
input_tensor = torch.randn(1, 3, 224, 224).to(device)
num_classes = 4

# ResNet-18

In [23]:
set_seed(42)

In [24]:
model_resnet = models.resnet18()
model_resnet.fc = nn.Linear(model_resnet.fc.in_features, num_classes)
model_resnet.load_state_dict(torch.load('/content/drive/MyDrive/res_best_model.pt', map_location=device))
model_resnet.to(device)
model_resnet.eval()

<ipython-input-24-d617ec6f750a>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_resnet.load_state_dict(torch.load('/content/drive/MyDrive/res_best_model.pt', map_loca

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [25]:
set_seed(42)

In [26]:
flops_resnet, params_resnet = profile(model_resnet, inputs=(input_tensor,), verbose=False)
model_resnet_size = os.path.getsize('/content/drive/MyDrive/res_best_model.pt') / (1024 * 1024)
start_time = time.time()
with torch.no_grad():
    for _ in range(10):
        outputs = model_resnet(input_tensor)
end_time = time.time()
avg_time_resnet = (end_time - start_time) / 10

print(f"ResNet18 - FLOPs: {flops_resnet:.2f}, Parameters: {params_resnet}, Model Size: {model_resnet_size:.2f} MB, Avg Time per Batch: {avg_time_resnet:.5f} s")

ResNet18 - FLOPs: 1823523840.00, Parameters: 11178564.0, Model Size: 42.72 MB, Avg Time per Batch: 0.00363 s


# AlexNet

In [27]:
model_alexnet = models.alexnet()
model_alexnet.classifier[6] = nn.Linear(model_alexnet.classifier[6].in_features, num_classes)
model_alexnet.load_state_dict(torch.load('/content/drive/MyDrive/alexnet_best_model.pt'))
model_alexnet.to(device)
model_alexnet.eval()

<ipython-input-27-c3db0e274f29>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_alexnet.load_state_dict(torch.load('/content/drive/MyDrive/alexnet_best_model.pt'))


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [28]:
set_seed(42)

In [29]:
flops_alexnet, params_alexnet = profile(model_alexnet, inputs=(input_tensor,), verbose=False)
model_alexnet_size = os.path.getsize('/content/drive/MyDrive/alexnet_best_model.pt') / (1024 * 1024)
start_time = time.time()
with torch.no_grad():
    for _ in range(10):
        outputs = model_alexnet(input_tensor)
end_time = time.time()
avg_time_alexnet = (end_time - start_time) / 10

print(f"AlexNet - FLOPs: {flops_alexnet:.2f}, Parameters: {params_alexnet}, Model Size: {model_alexnet_size:.2f} MB, Avg Time per Batch: {avg_time_alexnet:.5f} s")

AlexNet - FLOPs: 710127296.00, Parameters: 57020228.0, Model Size: 217.52 MB, Avg Time per Batch: 0.00091 s


# MobileNet

In [30]:
model_mobilenet = models.mobilenet_v2()
model_mobilenet.classifier[1] = nn.Linear(model_mobilenet.classifier[1].in_features, num_classes)
model_mobilenet.load_state_dict(torch.load('/content/drive/MyDrive/mobile_best_model.pt'))
model_mobilenet.to(device)
model_mobilenet.eval()

<ipython-input-30-116850ce6194>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_mobilenet.load_state_dict(torch.load('/content/drive/MyDrive/mobile_best_model.pt'))


MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [31]:
set_seed(42)

In [32]:
flops_mobilenet, params_mobilenet = profile(model_mobilenet, inputs=(input_tensor,), verbose=False)
model_mobilenet_size = os.path.getsize('/content/drive/MyDrive/mobile_best_model.pt') / (1024 * 1024)

start_time = time.time()
with torch.no_grad():
    for _ in range(10):
        outputs = model_mobilenet(input_tensor)
end_time = time.time()
avg_time_mobilenet = (end_time - start_time) / 10

print(f"MobileNetV2 - FLOPs: {flops_mobilenet:.2f}, Parameters: {params_mobilenet}, Model Size: {model_mobilenet_size:.2f} MB, Avg Time per Batch: {avg_time_mobilenet:.5f} s")

MobileNetV2 - FLOPs: 326211840.00, Parameters: 2228996.0, Model Size: 8.74 MB, Avg Time per Batch: 0.00669 s


In [33]:
import pandas as pd

In [34]:
all_results = [
    {'Model': 'ResNet18', 'FLOPs': flops_resnet, 'Parameters': params_resnet, 'Model Size (MB)': model_resnet_size, 'Ave Time per Batch (s)': avg_time_mobilenet},
    {'Model': 'AlexNet', 'FLOPs': flops_alexnet, 'Parameters': params_alexnet, 'Model Size (MB)': model_alexnet_size, 'Ave Time per Batch (s)': avg_time_mobilenet},
    {'Model': 'MobileNetV2', 'FLOPs': flops_mobilenet, 'Parameters': params_mobilenet, 'Model Size (MB)': model_mobilenet_size, 'Ave Time per Batch (s)': avg_time_mobilenet}
]

In [35]:
all_results_df = pd.DataFrame(all_results)
all_results_df.to_csv('model_performance_multiple.csv', index=False)
print("All model results saved to 'model_performance_multiple.csv'")

All model results saved to 'model_performance_multiple.csv'
